In [1]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from IPython.display import HTML

import thicket as th
from thicket.model_extrap import ExtrapInterface

display(HTML("<style>.container { width:80% !important; }</style>"))

# display all columns from pandas data frames
pd.options.display.max_columns = None

data = "../../data/LULESH/exclusive/weak"
thicket = th.Thicket.from_caliperreader(data)


<IPython.core.display.Javascript object>

In [2]:
#thicket.metadata

Instatiate an Extra-P interface to create performance models and more.

In [3]:
extrap_interface = ExtrapInterface()

To show the available modelers from Extra-P that can be used for modeling.

In [4]:
extrap_interface.print_modelers()

Available Extra-P Modeler: ['refining', 'basic', 'multi-parameter', 'default']


To display the available options for a specific modeler, e.g., the default modeler.

In [5]:
extrap_interface.print_modeler_options("default")

Modeler Options
--------------
allow_log_terms	 BOOL	 Allows models with logarithmic terms
use_crossvalidation	 BOOL	 Enables cross-validation
compare_with_RSS	 BOOL	 If enabled the models are compared using their residual sum of squares (RSS) instead of their symmetric mean absolute percentage error (SMAPE)
poly_exponents	 STR	 Set of polynomial exponents. Use comma separated list.
log_exponents	 STR	 Set of logarithmic exponents. Use comma separated list.
retain_default_exponents	 BOOL	 If set the default exponents are added to the given ones.
force_combination_exponents	 BOOL	 If set the exact combination of exponents is forced.
allow_negative_exponents	 BOOL	 If set adds neagtive exponents for strong scaling.



To create performance models use the create_models() function of the Extra-P interface class like shown in the example below. You have to pass the thicket object, the parameters and metrics you want to model, and a name for the model configuration you are going to create when running the create_models() function. Extra-P will internally use the name you provided as a unique identifier for your modeling experiments. This will come in handy when you experiment with different modeling parameters, metrics, and modeler configurations.

In [6]:
extrap_interface.create_models(thicket, 
                               parameters=[
                                   "jobsize",
                                ], 
                               metrics=[
                                   "Avg time/rank (exc)",
                                   ],
                               model_name="config1")

/usr/WS2/ritter5/thicket/thicket/model_extrap.py:1256: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '$-1.039\times10^{−3}+4.453\times10^{−4}\cdot \log_2{p}^{1}$' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tht.statsframe.dataframe.at[


If you want to explore Extra-P's modeling capabilities you can try to use a different modeler and see the difference in models it will return. To change the model generator Extra-P will use specify the name of the modeler as shown below. Here we use the refining modeler instead, which automatically adjusts the search space of the models for the user. Therefore, there is no need to specify the exponents Extra-P should use for modeling.
Furthermore, this example shows how to create models for several parameters and metrics.
To create models for several parameters or metrics simply provide them as a list of string values.

There are several additional parameters to this method. For example, one can use the parameter `use_median=True` to switch between using the median and mean values of the measured performance metrics values of a measurement point (application configuration) for modeling.

The `add_stats=True` parameter let's you specify if you want to have Extra-P's statistical values extended to the dataframe object of thicket. These values are internally used by Extra-P to decide which of the found hypotheses will be the best model for a certain node/kernel and metric.

In [7]:
extrap_interface.create_models(thicket, 
                               parameters=[
                                   "jobsize"
                                ], 
                               metrics=[
                                   "Avg time/rank (exc)",
                                   "Avg time/rank"
                                   ], 
                               use_median=True,
                               modeler="refining",
                               model_name="config2")

/usr/WS2/ritter5/thicket/thicket/model_extrap.py:1256: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '$1.699\times10^{−3}$' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tht.statsframe.dataframe.at[
/usr/WS2/ritter5/thicket/thicket/model_extrap.py:1256: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '$1.699\times10^{−3}$' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tht.statsframe.dataframe.at[


For advanced users, you can specific which options Extra-P should use when creating the models. This will influence the type of models you will get.

Define the modeler options that you want to use for modeling in the form of a dictonary and pass them to the create_models() function of the extrap interface.

Use the previously shown `print_modeler_options("default")` function to display which options are available for a specific modeler.

Make sure that the options you are setting are available for the modeler that you specified via `modeler=""`. If not the Extra-P interface will let you know and continue using the default options.

In [8]:
modeler_options = {'allow_log_terms': True,
                   'use_crossvalidation': True,
                   'compare_with_RSS': False,
                   'poly_exponents': [0,1,2,3,4,5],
                   'log_exponents': [0,1,2],
                   'retain_default_exponents': False,
                   'force_combination_exponents': True,
                   'allow_negative_exponents': False,
                   'allow_combinations_of_sums_and_products': False,
                   '#single_parameter_modeler': "default",
                   '#single_parameter_options': {'poly_exponents': "0,1,2,3,4,5",
                   'log_exponents': "0,1,2"},
                  }

extrap_interface.create_models(thicket, 
                               parameters=[
                                   "jobsize",
                                   "problem_size"
                                ], 
                               metrics=[
                                   "Avg time/rank (exc)",
                                   #"Avg time/rank"
                                   ], 
                               use_median=True,
                               modeler="multi-parameter",
                               model_name="config3",
                               modeler_options=modeler_options)

In [9]:
pd.set_option('display.max_colwidth', None)
thicket.statsframe.dataframe

config1  \
                                                                                            name   
node                                                                                               
{'name': 'MPI_Barrier', 'type': 'function'}                                          MPI_Barrier   
{'name': 'MPI_Comm_dup', 'type': 'function'}                                        MPI_Comm_dup   
{'name': 'MPI_Finalize', 'type': 'function'}                                        MPI_Finalize   
{'name': 'MPI_Finalized', 'type': 'function'}                                      MPI_Finalized   
{'name': 'MPI_Initialized', 'type': 'function'}                                  MPI_Initialized   
{'name': 'main', 'type': 'function'}                                                        main   
{'name': 'CommRecv', 'type': 'function'}                                                CommRecv   
{'name': 'MPI_Irecv', 'type': 'function'}                                              MPI_Irecv   
{'name': 'CommSBN', 'type': 'function'}                                                  CommSBN   
{'name': 'MPI_Wait', 'type': 'function'}                                                MPI_Wait   
{'name': 'CommSend', 'type': 'function'}                                                CommSend   
{'name': 'MPI_Isend', 'type': 'function'}                                              MPI_Isend   
{'name': 'MPI_Waitall', 'type': 'function'}                                          MPI_Waitall   
{'name': 'MPI_Barrier', 'type': 'function'}                                          MPI_Barrier   
{'name': 'MPI_Reduce', 'type': 'function'}                                            MPI_Reduce   
{'name': 'lulesh.cycle', 'type': 'function'}                                        lulesh.cycle   
{'name': 'LagrangeLeapFrog', 'type': 'function'}                                LagrangeLeapFrog   
{'name': 'CalcTimeConstraintsForElems', 'type': 'function'}          CalcTimeConstraintsForElems   
{'name': 'LagrangeElements', 'type': 'function'}                                LagrangeElements   
{'name': 'ApplyMaterialPropertiesForElems', 'type': 'function'}  ApplyMaterialPropertiesForElems   
{'name': 'EvalEOSForElems', 'type': 'function'}                                  EvalEOSForElems   
{'name': 'CalcEnergyForElems', 'type': 'function'}                            CalcEnergyForElems   
{'name': 'CalcLagrangeElements', 'type': 'function'}                        CalcLagrangeElements   
{'name': 'CalcKinematicsForElems', 'type': 'function'}                    CalcKinematicsForElems   
{'name': 'CalcQForElems', 'type': 'function'}                                      CalcQForElems   
{'name': 'CalcMonotonicQForElems', 'type': 'function'}                    CalcMonotonicQForElems   
{'name': 'CommMonoQ', 'type': 'function'}                                              CommMonoQ   
{'name': 'MPI_Wait', 'type': 'function'}                                                MPI_Wait   
{'name': 'CommRecv', 'type': 'function'}                                                CommRecv   
{'name': 'MPI_Irecv', 'type': 'function'}                                              MPI_Irecv   
{'name': 'CommSend', 'type': 'function'}                                                CommSend   
{'name': 'MPI_Isend', 'type': 'function'}                                              MPI_Isend   
{'name': 'MPI_Waitall', 'type': 'function'}                                          MPI_Waitall   
{'name': 'LagrangeNodal', 'type': 'function'}                                      LagrangeNodal   
{'name': 'CalcForceForNodes', 'type': 'function'}                              CalcForceForNodes   
{'name': 'CalcVolumeForceForElems', 'type': 'function'}                  CalcVolumeForceForElems   
{'name': 'CalcHourglassControlForElems', 'type': 'function'}        CalcHourglassControlForElems   
{'name': 'CalcFBHourglassForceForElems', 'type': 'function'}        CalcFBHourglassForceF

In [ ]:
#t_ens.statsframe.dataframe["Avg time/rank (exc)_extrap-model"].iloc[0].mdl.hypothesis.function

In [ ]:
model_obj = t_ens.statsframe.dataframe.at[t_ens.statsframe.dataframe.index[0], "Avg time/rank (exc)_extrap-model"]

plt.clf()
fig, ax = model_obj.display(show_mean=True, show_median=True, 
                            show_min_max=True, RSS=True, 
                            AR2=True, show_opt_scaling=True,
                            opt_scaling_func="1.5*p**1*log2(p)**1")
 
# Plot graph
plt.show()


In [ ]:
with pd.option_context("display.max_colwidth", 1):
    display(HTML(mdl.to_html(show_mean=True, show_median=True, 
                            show_min_max=True, RSS=True, 
                            AR2=True, show_opt_scaling=False)))


In [ ]:
mdl.componentize_statsframe()

xp_comp_df = t_ens.statsframe.dataframe

xp_comp_df

In [ ]:
#mdl.complexity_statsframe(eval_targets=[[512,70],[1024,80]])
mdl.complexity_statsframe(eval_targets=[[512],[1024]])

xp_comp_df = t_ens.statsframe.dataframe

xp_comp_df

In [ ]:
# sort by growth rank to identify bottlenecks
#x = xp_comp_df.sort_values(by=["Avg time/rank (exc)_extrap-model_complexity_(512,70)"])
x = xp_comp_df.sort_values(by=["Avg time/rank (exc)_extrap-model_complexity_(512)"])
x

In [ ]:
#x = xp_comp_df.groupby("Avg time/rank (exc)_extrap-model_complexity_(512,70)")
x = xp_comp_df.groupby("Avg time/rank (exc)_extrap-model_complexity_(512)")
for key, value in x.groups.items():
    print(key)
    print(value)
